<a href="https://colab.research.google.com/github/mcstllns/Semana-de-la-Ciencia/blob/main/Demo_Semana_de_la_Ciencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://biblioteca.ucm.es/data/cont/docs/60-2016-09-20-Marca%20UCM%20Secundaria%20Monocromo%20Tramado.jpg)

\
# Uso de la Inteligencia Artificial Generativa en Psicología

**Profesor**: Miguel A. Castellanos (mcastellanos@psi.ucm.es)

\
Este notebook, y los powerpoints relacionados, se encuentran en esta carpeta de github ["Demo para la Semana de la Ciencia"]().

\
En este cuaderno vamos a crear un mini ejemplo de como se pueden usar las IAs generativas para crear un test proyectivo (como el test voight-kampff de la película) que sea a su vez objetivo y científico.

\
Este tipo de test no existe aún, ni existirá durante un tiempo, simplemente es un ejemplo de lo que esperamos que ocurra en el futuro en el área de Psicometría. La mayoría de las universidades y centros de investigación estamos trabajando en esta línea.

\
El principal motivo por el que hoy en día no es seguro usar IAs para la evaluación es porque su propiedades métricas (qué miden, cómo lo miden, en qué grado, qué intensidad, etc.) son desconocidas y no son directamente controlables. Con el tiempo estoy seguro de que estos modelos sí serán plausibles.

\
En esta demo vamos a utilizar diversos trasformers para generar items automáticamente y analizarlos las respuestas de un sujeto. Además, podemos utilizar imágenes tomadas de los respondientes para analizar sus expresiones mientras contestan y asociarlas con el test.

\
Aunque algunos modelos pueden funcionar razonablemente en castellano es mejor trabajar con ellos directamente en inglés, si da pereza escribir directamente os recomiendo usar [***DeepL.com***](https://www.deepl.com/).

\
La demo está dividida en tres partes:
- Vamos a generar items de forma automática.
- Vamos a analizar la respuesta dada por un respondiente.
- Vamos a cargar una imagen y a hacer preguntas a la IA sobre ella.

Y para ello utilizaremos los siguientes modelos transformers:
- ***GPT-2***: Para la generación de los items.
- ***distilroberta***: Para la clasificación de las emociones.
- ***blip-vqa***: Para el procesamiento de imágenes.


\
Para ejecutar esta demo tienes que cargar este fichero en el servicio [colab](https://colab.research.google.com/) de Google y tener una cuenta de Gmail.



## 1. Generación de ítems

Esta parte del código está diseñada para completar sentencias, por ejemplo, yo empiezo diciendo "An unhappy person talk about" y la IA debe generarme items que sea coherentes con ese inicio, por ejemplo:

- "committing suicide at some point in time"
- "how horrible life is"
- "being friendless and lonely in life"
- etc.

Si controlamos la creación de ítems podemos conseguir test absolutamente personalizados y adpatativos que no se **"*quemen*"** nunca.

Si probamos a sustituir el ***prompt*** podemos generar items siempre nuevos para medir distintos rasgos en distintos niveles.


In [ ]:
# Load libraries and functions
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "gpt2-medium"  # Choose a version of GPT-2, e.g., "gpt2", "gpt2-medium", "gpt2-large", etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:

# Define the prompt you want to complete



# ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
prompt = "An unhappy person talks about"
# ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑



# Encode the input and move tensors to the model's device (CPU/GPU)
inputs = tokenizer(prompt, return_tensors="pt")
inputs = {key: value.to(model.device) for key, value in inputs.items()}

# Generate text
with torch.no_grad():
    output = model.generate(**inputs, max_length=50, do_sample=True, temperature=0.7)

# Decode the output and print the result
completion = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
print("Generated ITEM: \n\n")
print(completion)

## 2. Análisis de respuestas

Escribe una respuesta que sea coherente con el ítem que has generado. Puede ser todo lo larga que quieras pero es mejor si es corta.

In [ ]:
from transformers import pipeline

# Crea un pipeline para clasificación de emociones
clasificador_emociones = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)

# Texto para analizar

# ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
texto = "I am very happy because I have been told that I am handsome."
# ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑



# Clasifica la emoción principal en el texto
emocion = clasificador_emociones(texto)


In [ ]:
# Imprime la emoción detectada
print("Emoción principal detectada:")
print(emocion[0])

## 3. Análisis de imágenes

Este código va a generarte una ventana para que desde la cámara de tu portatil tomes una imagen y hagas preguntas sobre ella.

Al ejecutarlo te aparecerá un enlace que tienes que pinchar para que se abra la ventana.

In [ ]:
from transformers import BlipProcessor, BlipForQuestionAnswering
import gradio as gr

procesador = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
modelo = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

def question_fn(image, question):
  salida = modelo.generate(**procesador(images=image, text=question, return_tensors="pt", padding=True))
  respuesta = procesador.decode(salida[0], skip_special_tokens=True)
  return respuesta

# Define the Gradio interface
iface = gr.Interface(
    fn=question_fn,
    inputs=[gr.Image(type="pil"), gr.Textbox(label="Question")],
    outputs=gr.Textbox(label="Answer"),
    title="Visual Question Answering",
    description="Upload an image and ask a question related to the image. The AI will try to answer it."
)

In [ ]:
# Launch the interface
iface.launch()

## Ya hemos terminado!!

Muchas gracias por tu tiempo, espero que te haya parecido interesante y hayas aprendido algo sobre la Psicología, la Psicometría y la Psicología matemática.

Si quieres llevarte esta demo y los powerpoint a casa accede a esta dirección web o haz una foto a este QR.


![](https://github.com/mcstllns/Semana-de-la-Ciencia/blob/main/QR.png?raw=true)